In [1]:
import os
import pandas as pd

In [2]:
from census import Census
from us import states

# Note, you must obtain a census key and create a system environment variable called CENSUS_KEY that stores the key value
# You may need to completely restart anaconda/jupyter/VS to get it to recognize the new variable
c = Census(os.environ['CENSUS_KEY'])

census_year = 2023

In [3]:
year = '2023'

In [4]:
def get_acs_data_place(table):
    results_df = pd.DataFrame()

    df = pd.DataFrame(
        c.acs5.get(table, geo={'for': 'place:*',
                   'in': 'state:{}'.format(states.WA.fips)})
    )

    return df

# Place

In [5]:
place_name_df = pd.DataFrame(c.sf1.get('NAME', geo={'for': 'place:*',
                       'in': 'state:{}'.format(states.WA.fips)})
            )
place_name_df.drop('state', axis=1, inplace=True)
# place_name_df

In [6]:

acs_dict = {
#     'B08601_001E': 'Total',
    'B08601_003E': 'sov',
#     'B08601_004E': 'Carpooled',
    'B08601_010E': 'transit',
    'B08601_019E': 'walk_and_bike',
#     'B08601_020E': 'Other',
    'B08601_021E': 'worked_at_home',
}

df_place = pd.DataFrame()

for table_id, mode in acs_dict.items():
    df = get_acs_data_place(table_id)
    df['mode'] = mode
    df.rename(columns={table_id: 'trips'}, inplace=True)
    df_place = pd.concat([df_place, df])

In [7]:
df_place = df_place.merge(place_name_df, on='place', how='left')

In [11]:
# Only select places needed in region.
# Use existing table as template
df_template = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\hold\acs_commute_mode_by_workplace_geog.csv')
df_template = df_template[df_template['year'] != int(year)]
df_template['geog_name'].unique()

array(['King County, Washington', 'Kitsap County, Washington',
       'Pierce County, Washington', 'Snohomish County, Washington',
       'Auburn city, Washington', 'Bellevue city, Washington',
       'Bellingham city, Washington', 'Everett city, Washington',
       'Federal Way city, Washington', 'Kent city, Washington',
       'Kirkland city, Washington', 'Renton city, Washington',
       'Seattle city, Washington', 'Tacoma city, Washington',
       'Redmond city, Washington'], dtype=object)

In [12]:
# Note that we are missing many places here;
# The older version must have been trimmed down to only include a select set of places
df_place = df_place[df_place['NAME'].isin(df_template['geog_name'].unique())]

In [13]:
df_place.rename(columns={'NAME': 'geog_name'}, inplace=True)

In [14]:
df_place['year'] = year
df_place['geography'] = 'place'
df_place = df_place[['geog_name','mode','trips','geography','year']]

# County

In [15]:
def get_acs_data(table):
    results_df = pd.DataFrame()
    county_map = {'033': 'King',
                   '035': 'Kitsap',
                   '053': 'Pierce',
                   '061': 'Snohomish'
                  }
    df = pd.DataFrame(
        c.acs5.get(table, geo={'for': 'county:*',
                   'in': 'state:{}'.format(states.WA.fips)})
    )
    df['county_name'] = df['county'].map(county_map)
    df = df[~df['county_name'].isnull()]

    return df

In [16]:
df_county = pd.DataFrame()

for table_id, mode in acs_dict.items():
    df = get_acs_data(table_id)
    df['mode'] = mode
    df.rename(columns={table_id: 'trips'}, inplace=True)
    df_county = pd.concat([df_county, df])

In [17]:
df_county.rename(columns={'county_name': 'geog_name'}, inplace=True)

In [18]:
df_county['year'] = year
df_county['geography'] = 'county'
df_county = df_county[['geog_name','mode','trips','geography','year']]

In [19]:
df_county

,geog_name,mode,trips,geography,year
16,King,sov,783421.0,county,2023
17,Kitsap,sov,84641.0,county,2023
26,Pierce,sov,287932.0,county,2023
30,Snohomish,sov,223917.0,county,2023
16,King,transit,137905.0,county,2023
17,Kitsap,transit,3319.0,county,2023
26,Pierce,transit,4886.0,county,2023
30,Snohomish,transit,5355.0,county,2023
16,King,walk_and_bike,56853.0,county,2023
17,Kitsap,walk_and_bike,7032.0,county,2023


In [20]:
final_df = pd.concat([df_county, df_place, df_template])

In [21]:
final_df = final_df[['field1','geog_name','mode','trips','geography','year']]

In [22]:
final_df['field1'] = range(1,len(final_df)+1)
final_df.to_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\acs_commute_mode_by_workplace_geog.csv',
               index=False)

In [23]:
final_df

,field1,geog_name,mode,trips,geography,year
16,1,King,sov,783421.0,county,2023
17,2,Kitsap,sov,84641.0,county,2023
26,3,Pierce,sov,287932.0,county,2023
30,4,Snohomish,sov,223917.0,county,2023
16,5,King,transit,137905.0,county,2023
...,...,...,...,...,...,...
171,172,"Seattle city, Washington",walk_and_bike,74582.0,place,2018
172,173,"Tacoma city, Washington",sov,105163.0,place,2018
173,174,"Tacoma city, Washington",worked_at_home,7608.0,place,2018
174,175,"Tacoma city, Washington",transit,3199.0,place,2018
